**The Cholesky Decomposition**

Downloading matrix

In [1]:
import requests

matrix_id = "HB/bcsstk16"  # choose one of the IDs above
url = f"https://sparse.tamu.edu/mat/{matrix_id}.mat"
dest_filename = matrix_id.split("/")[-1] + ".mat"

# Download the file using requests
response = requests.get(url, stream=True)
response.raise_for_status()

with open(dest_filename, 'wb') as f:
    for chunk in response.iter_content(chunk_size=8192):
        f.write(chunk)

print(f"Downloaded {dest_filename}")

Downloaded bcsstk16.mat


Loading Matrix

In [2]:
import numpy as np
from scipy.io import loadmat
from scipy.sparse import csr_matrix

# Load the .mat file
data = loadmat("bcsstk16.mat")

# Access the matrix stored in the 'Problem' struct
A = data['Problem'][0, 0]['A']  # This is typically a sparse matrix already

# If A is not in CSR format, convert it
A_sparse = csr_matrix(A)

# Load RHS vector if available
if 'b' in data['Problem'][0, 0].dtype.names:
    b = data['Problem'][0, 0]['b'].flatten()
else:
    b = np.random.rand(A_sparse.shape[0])  # Generate dummy RHS

# Inspect matrix
print("Shape:", A_sparse.shape)
print("Non-zeros:", A_sparse.nnz)

Shape: (4884, 4884)
Non-zeros: 290378


3. The Cholesky Decomposition

    Matrix Requirement: Square, Symmetric,SPD

In [3]:
import numpy as np

def cholesky_decomposition(A):
    # Convert sparse to dense
    A = np.array(A.todense(), dtype=float)
    n = A.shape[0]
    L = np.zeros((n, n))

    # Double Loop (Row by Row, Column by Column)
    for i in range(n):
        for j in range(i+1):
            temp_sum = np.dot(L[i, :j], L[j, :j]) # compute partial sum
            if i == j: # Diagonal Case (i == j)
                val = A[i, i] - temp_sum
                if val <= 0:
                    raise ValueError(f"Matrix is not SPD at diagonal {i}")
                L[i, j] = np.sqrt(val)
            else: # Off‑Diagonal Case (i ≠ j
                L[i, j] = (A[i, j] - temp_sum) / L[j, j]
    return L

# usage
L = cholesky_decomposition(A_sparse)

print("Lower-triangular matrix L:\n", L)

# Verify correctness
A_dense = np.array(A_sparse.todense(), dtype=float)
residual = np.linalg.norm(A_dense - L @ L.T)
print("\nResidual norm ||A - LL^T||:", residual)

Lower-triangular matrix L:
 [[ 1.68985169e+04  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 1.57804775e+03  1.55906292e+04  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 ...
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  6.19608083e+03
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ... -5.81332823e+03
   8.39712561e+03  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  1.00000000e+00]]

Residual norm ||A - LL^T||: 6.854881429321971e-06
